In [1]:
import numpy as np
import matplotlib.pyplot as mpl

In [2]:
train_X = np.loadtxt('xy/x.csv', delimiter=',')
train_Y = np.loadtxt('xy/y.csv', delimiter=',')
theta_1 = np.loadtxt('theta/theta1.csv', delimiter=',')
theta_2 = np.loadtxt('theta/theta2.csv', delimiter=',')

In [3]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))   
    return s

In [4]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(2) # Recordemos que los pesos /tetha/W se inicializan en random
    
    #W1 = np.random.randn(n_h, n_x) * 0.01
    #W2 = np.random.randn(n_y, n_h ) * 0.01
    W1 = theta_1
    W2 = theta_2


    #assert (W1.shape == (n_h, n_x))
    #assert (W2.shape == (n_y, n_h ))
    
    parameters = {"W1": W1,                  
                  "W2": W2}
    return W1,W2
    

In [17]:
#Forward
def forward(X,W1, W2):
     
    m = X.shape[0]
    A1 = np.c_[np.ones((m,1)),X] #Añadimos el bías (5000,401)
    Z2 = np.dot(A1, W1.T)#(5000,401) * (401,25) = (5000,25)
    A2 = sigmoid(Z2)# (5000,25)
    A2 = np.c_[np.ones((Z2.shape[0],1)),A2] #Añadiendo una columna (5000,26)
    Z3 = np.dot(A2,W2.T) #(5000,26) * (26,10) = 5000,10)
    A3 = sigmoid(Z3)#(5000,10) --nuestra predicción
    
    cache = {"A1": A1,
             "Z2": Z2,
             "A2": A2,
             "Z3": Z3,
             "A3": A3}
    
    return A3,cache

In [6]:
#Calculamos el Costo

def compute_cost(A3, Y):
    m = Y.shape[0] #(5000,10)
    cost = (- 1 / m) * np.sum(np.sum(Y * np.log(A3) + (1 - Y) * (np.log(1 - A3)), axis=1), axis=0)
    return cost


In [7]:
def sigmoidGradient(z):
    gz = sigmoid(z)
    return gz * (1-gz);

In [20]:
#Cálculo de la Gradiente
#Esta funcion retornará el costo y el gradiente
def compute_gradient_reg(cache,W2,m, yNew):
    
    a1 = cache["A1"] #(5000,401)
    z2 = cache["Z2"] #(5000,26)
    a2 = cache["A2"] #(5000,26)
    z3 = cache["Z3"] #(5000,10)
    a3 = cache["A3"] #(5000,10)
        
    d3 = a3 - yNew #(5000,10)
    d2 = np.dot(d3,W2) * sigmoidGradient(np.c_[np.ones((z2.shape[0],1)),z2])#sigmoide again
    #d2 = np.dot(d3,W2) * sigmoidGradient(z2)#sigmoide again
    #print(sigmoidGradient(z2).shape)
    #print("d2: ", d2[0][0])
    d2 = d2[:,1:]  #(5000,25)
    
    delta1 = np.dot(d2.T,a1) #(25,5000).(5000,401) = (25,401)
    delta2 = np.dot(d3.T,a2) #(10,5000).(5000,26) = (10,26)
    
    grad1 = delta1/m
    grad2 = delta2/m
    
    return grad1,grad2

In [18]:
# GRADED FUNCTION: nn_model

def nn_model(X, Y, n_h, num_iterations):

    np.random.seed(3)
    n_x = X.shape[1]+1
    n_y = Y.shape[1] 
    m = X.shape[0]
    learning_rate = 1.5
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    w1,w2 = initialize_parameters(n_x, n_h, n_y)
    
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A3, cache = forward(X, w1, w2)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A3, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grad1,grad2 = compute_gradient_reg(cache, w2, m, Y)
        
        w1 = w1 - learning_rate * grad1
        w2 = w2 - learning_rate * grad2
        
        if i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    return w1,w2


In [10]:
zeros = np.zeros((1,10))
tmp = np.zeros((1,10))
new_train_Y = np.zeros((5000,10))
for i in range(0,len(train_Y)):
    zeros[0][int(train_Y[i]-1)] = 1
    new_train_Y[i] = zeros
    zeros = tmp        
        
print(new_train_Y[1097])

[1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]


In [11]:
yNew = np.zeros((5000,10))
print(train_Y[1097]%11)
for i in range(train_Y.shape[0]):
    aux = train_Y[i]%11 - 1
    yNew[i,int(aux)] = 1
print(yNew[0])

2.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [21]:
w1,w2 = nn_model(train_X, yNew, 25, 2000)

Cost after iteration 0: 0.287629
Cost after iteration 100: 0.247440
Cost after iteration 200: 0.220436
Cost after iteration 300: 0.199926
Cost after iteration 400: 0.183406
Cost after iteration 500: 0.169613
Cost after iteration 600: 0.157815
Cost after iteration 700: 0.147544
Cost after iteration 800: 0.138485
Cost after iteration 900: 0.130413
Cost after iteration 1000: 0.123161
Cost after iteration 1100: 0.116603
Cost after iteration 1200: 0.110636
Cost after iteration 1300: 0.105179
Cost after iteration 1400: 0.100164
Cost after iteration 1500: 0.095535
Cost after iteration 1600: 0.091248
Cost after iteration 1700: 0.087271
Cost after iteration 1800: 0.083577
Cost after iteration 1900: 0.080138


In [15]:
_A2,_cache = forward(train_X,w1,w2)
print(_A2[1097])

NameError: name 'w1' is not defined

In [15]:
mat_cross = np.zeros((10,10))
prediccion = np.zeros((5000,1))
for i in range(0,len(_A2)):
    for j in range(0,len(_A2[i])):
        if _A2[i][j] >0.5:
            prediccion[i][0] = j + 1
            
for i in range(0,len(prediccion)):
    _j = int(train_Y[i]-1) #valor real
    _i = int(prediccion[i][0]-1) #valor predicho
    mat_cross[_i][_j] += 1
    
print(mat_cross)

[[490.   1.   1.   1.   0.   0.   0.   0.   1.   0.]
 [  0. 482.   0.   0.   0.   0.   1.   0.   0.   0.]
 [  0.   0. 478.   0.   1.   0.   0.   1.   2.   0.]
 [  0.   0.   0. 487.   0.   0.   0.   0.   0.   0.]
 [  1.   0.   3.   0. 488.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   1.   0. 491.   0.   0.   0.   0.]
 [  1.   2.   3.   0.   0.   0. 485.   0.   1.   0.]
 [  0.   0.   1.   0.   0.   0.   0. 492.   0.   0.]
 [  0.   0.   1.   4.   1.   0.   0.   0. 481.   1.]
 [  8.  15.  13.   7.  10.   9.  14.   7.  14. 499.]]
